# LangChain: Evaluation

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation
* LangChain evaluation platform

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Create our QandA application

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [4]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding='utf-8')
data = loader.load()

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


c:\Users\jefer\Documents\Livros\LLMs\LangChain-for-LLM-Application-Development\env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [6]:
index = VectorstoreIndexCreator(
    embedding=embeddings,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

c:\Users\jefer\Documents\Livros\LLMs\LangChain-for-LLM-Application-Development\env\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [7]:
#llm = ChatOpenAI(temperature = 0.0, model=llm_model)
from langchain_groq import ChatGroq

llm = ChatGroq(
    model= "llama-3.2-11b-vision-preview",
    #model = "llama3-groq-70b-8192-tool-use-preview",
    #model = "llama-3.2-90b-vision-preview",
    #model="gemma2-9b-it",
    #model="gemma-7b-it",
    #model="llama3-groq-8b-8192-tool-use-preview",
    #model="distil-whisper-large-v3-en", ## huggingface 
    #model="llama-3.1-70b-versatile",
    #model="llava-v1.5-7b-4096-preview",
    #model="llama3-70b-8192",
    #model="whisper-large-v3",
    temperature=0
)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Coming up with test datapoints

In [8]:
data[10]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [9]:
data[11]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

### Hard-coded examples

In [35]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated examples

In [36]:
from langchain.evaluation.qa import QAGenerateChain


In [37]:
#example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm))
example_gen_chain = QAGenerateChain.from_llm(llm)

In [38]:
# the warning below can be safely ignored

In [39]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

c:\Users\jefer\Documents\Livros\LLMs\LangChain-for-LLM-Application-Development\env\lib\site-packages\langchain\chains\llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [40]:
new_examples = [list(itens.values())[0] for itens in new_examples]

In [41]:
data[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

### Combine examples

In [42]:
examples += new_examples

In [43]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What type of material is used for the innersole of the Women's Campside Oxfords, which also features antimicrobial odor control?",
  'answer': 'EVA (Ethylene-Vinyl Acetate) material with Cleansport NXT antimicrobial odor control.'},
 {'query': 'What percentage of the polyester fabric used in the Recycled Waterhog Dog Mat is made from recycled materials?',
  'answer': '94%'},
 {'query': "What type of fabric is used in the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece, which provides the highest rated sun protection possible?",
  'answer': 'UPF 50+ rated fabric.'},
 {'query': 'What percentage of recycled nylon is used in the lining of the Refresh Swimwear, V-Neck Tankini Contrasts?',
  'answer': '90%'},
 {'query': 'What type of fabric is us

In [63]:
qa.run(examples[1]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'

## Manual Evaluation

In [44]:
import langchain
langchain.debug = True

In [45]:
qa.run(examples[0]["query"])

C:\Users\jefer\AppData\Local\Temp\ipykernel_15316\1223946598.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(examples[0]["query"])


[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 73\nname: Cozy Cuddles Knit Pullover Set\ndescription: Perfect for lounging, this knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out. \n\nSize & Fit \nPants are Favorite Fit: Sits lower on the waist. \nRelaxed Fit: Our most generous fit sits farthest from the body. \n\nFabric & Care \nIn the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features \nRelaxed fit top with raglan sleeves and rounded hem. \nPull-on pants have 

'Yes, the Cozy Comfort Pullover Set has side pockets on the pull-on pants.'

In [46]:
# Turn off the debug mode
langchain.debug = False

## LLM assisted evaluation

In [47]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What type of material is used for the innersole of the Women's Campside Oxfords, which also features antimicrobial odor control?",
  'answer': 'EVA (Ethylene-Vinyl Acetate) material with Cleansport NXT antimicrobial odor control.'},
 {'query': 'What percentage of the polyester fabric used in the Recycled Waterhog Dog Mat is made from recycled materials?',
  'answer': '94%'},
 {'query': "What type of fabric is used in the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece, which provides the highest rated sun protection possible?",
  'answer': 'UPF 50+ rated fabric.'},
 {'query': 'What percentage of recycled nylon is used in the lining of the Refresh Swimwear, V-Neck Tankini Contrasts?',
  'answer': '90%'},
 {'query': 'What type of fabric is us

In [48]:
predictions = qa.apply(examples)

C:\Users\jefer\AppData\Local\Temp\ipykernel_15316\1205324748.py:1: LangChainDeprecationWarning: The method `Chain.apply` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~batch` instead.
  predictions = qa.apply(examples)




> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [49]:
from langchain.evaluation.qa import QAEvalChain

In [50]:
#llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [51]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [53]:
graded_outputs

[{'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': "GRADE: CORRECT\n\nThe student's answer accurately identifies the fabric as UPF 50+ rated, which is the key point of the question. The additional information about the fabric being four-way-stretch and chlorine-resistant is not necessary to answer the question, but it does not conflict with the true answer."},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'}]

In [54]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set has side pockets on the pull-on pants.
Predicted Grade: GRADE: CORRECT

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade: GRADE: CORRECT

Example 2:
Question: What type of material is used for the innersole of the Women's Campside Oxfords, which also features antimicrobial odor control?
Real Answer: EVA (Ethylene-Vinyl Acetate) material with Cleansport NXT antimicrobial odor control.
Predicted Answer: The innersole of the Women's Campside Oxfords features a Comfortable EVA (Ethylene-Vinyl Acetate) innersole with Cleansport NXT antimicrobial odor control.
Predicted Grade: GRADE: CORRECT

Example 3:
Question: What percentage of the polyester f

In [55]:
graded_outputs[0]

{'results': 'GRADE: CORRECT'}

## LangChain evaluation platform

The LangChain evaluation platform, LangChain Plus, can be accessed here https://www.langchain.plus/.  
Use the invite code `lang_learners_2023`

Reminder: Download your notebook to you local computer to save your work.